In [1]:
#-*- coding : utf-8-*-
# coding:unicode_escape
import warnings
warnings.filterwarnings("ignore")

# the location of R (used for the mclust clustering)
import os
os.environ['R_HOME'] = "D:\\anaconda\envs\STAligner\Lib\R"
os.environ['R_USER'] = "D:\\anaconda\envs\STAligner\Lib\site-packages\rpy2"


os.chdir("D:/bio/SPIRAL/SPIRAL-main") 

import numpy as np
import argparse

import pandas as pd
from sklearn.decomposition import PCA
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import umap.umap_ as umap
import time
import torch

from spiral.main import SPIRAL_integration
from spiral.layers import *
from spiral.utils import *
from spiral.CoordAlignment import CoordAlignment

In [2]:
dirs="G:/dataset/2_saggital/input/SPIRAL_input/"
sample_name=['anterior1', 'posterior1']
for sample in sample_name:
    df = pd.read_csv(dirs+sample+'_meta.csv', index_col=0)
    df2 = df[['batch', 'celltype_mapped_refined']]
    df2.to_csv(dirs+sample+'_label.csv')
    df3 = df[['x_global', 'y_global']]
    df3.to_csv(dirs+sample+'_coord.csv')

In [2]:
# dirs="G:/dataset/07-seqFish-SpatialMouseAtlas/input/SPIRAL_input/"
# sample_name=['embryo1-2', 'embryo1-5', 'embryo2-2','embryo2-5', 'embryo3-2', 'embryo3-5']
dirs="G:/dataset/2_saggital/input/SPIRAL_input/"
sample_name=['anterior1', 'posterior1']
samples=sample_name
SEP=','
net_cate='_KNN_'
rad=150
knn=6

N_WALKS=knn
WALK_LEN=1
N_WALK_LEN=knn
NUM_NEG=knn

feat_file=[]
edge_file=[]
meta_file=[]
coord_file=[]
flags=''
flags1=str(samples[0])
for i in range(1,len(samples)):
    flags1=flags1+'-'+str(samples[i])
for i in range(len(samples)):
    feat_file.append(dirs+samples[i]+"_features.csv")
    edge_file.append(dirs+samples[i]+"_edge_KNN_"+str(knn)+".csv")
    meta_file.append(dirs+samples[i]+"_label.csv")
    coord_file.append(dirs+samples[i]+"_coord.csv")
    flags=flags+'_'+str(samples[i])
                     
N=pd.read_csv(feat_file[0],header=0,index_col=0).shape[1]
if (len(samples)==2):
    M=1
else:
    M=len(samples)

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument('--seed', type=int, default=0, help='The seed of initialization.')
parser.add_argument('--AEdims', type=list, default=[N,[512],32], help='Dim of encoder.')
parser.add_argument('--AEdimsR', type=list, default=[32,[512],N], help='Dim of decoder.')
parser.add_argument('--GSdims', type=list, default=[512,32], help='Dim of GraphSAGE.')
parser.add_argument('--zdim', type=int, default=32, help='Dim of embedding.')
parser.add_argument('--znoise_dim', type=int, default=4, help='Dim of noise embedding.')
parser.add_argument('--CLdims', type=list, default=[4,[],M], help='Dim of classifier.')
parser.add_argument('--DIdims', type=list, default=[28,[32,16],M], help='Dim of discriminator.')
parser.add_argument('--beta', type=float, default=1.0, help='weight of GraphSAGE.')
parser.add_argument('--agg_class', type=str, default=MeanAggregator, help='Function of aggregator.')
parser.add_argument('--num_samples', type=str, default=knn, help='number of neighbors to sample.')

parser.add_argument('--N_WALKS', type=int, default=N_WALKS, help='number of walks of random work for postive pairs.')
parser.add_argument('--WALK_LEN', type=int, default=WALK_LEN, help='walk length of random work for postive pairs.')
parser.add_argument('--N_WALK_LEN', type=int, default=N_WALK_LEN, help='number of walks of random work for negative pairs.')
parser.add_argument('--NUM_NEG', type=int, default=NUM_NEG, help='number of negative pairs.')


parser.add_argument('--epochs', type=int, default=100, help='Number of epochs to train.')
parser.add_argument('--batch_size', type=int, default=1024, help='Size of batches to train.') ####512 for withon donor;1024 for across donor###
parser.add_argument('--lr', type=float, default=1e-3, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay.')
parser.add_argument('--alpha1', type=float, default=N, help='Weight of decoder loss.')
parser.add_argument('--alpha2', type=float, default=1, help='Weight of GraphSAGE loss.')
parser.add_argument('--alpha3', type=float, default=1, help='Weight of classifier loss.')
parser.add_argument('--alpha4', type=float, default=1, help='Weight of discriminator loss.')
parser.add_argument('--lamda', type=float, default=1, help='Weight of GRL.')
parser.add_argument('--Q', type=float, default=10, help='Weight negative loss for sage losss.')

params,unknown=parser.parse_known_args()

In [ ]:
def load_data(feat_file,edge_file,SEP,dist_aware=True,hvg_file=None):
    feat=pd.read_csv(feat_file[0],header=0,index_col=0,sep=SEP)
    edge=np.loadtxt(edge_file[0],dtype=str)
    batch={}
    for i in np.arange(feat.shape[0]):
        batch[i]=np.arange(feat.shape[0])
    if len(edge.shape)<2:
        edge=np.array([[i.split(":")[0],i.split(":")[1]] for i in edge])
    if len(feat_file)>1:
        for k in np.arange(1,len(feat_file)):
            a=pd.read_csv(feat_file[k],header=0,index_col=0,sep=SEP)
            for i in np.arange(feat.shape[0],a.shape[0]+feat.shape[0]):
                batch[i]= np.arange(feat.shape[0],a.shape[0]+feat.shape[0])
            if hvg_file is not None:
                a=a.loc[:,np.loadtxt(hvg_file,dtype=str)]
            feat=pd.concat((feat,a),axis=0)
            a=np.loadtxt(edge_file[k],dtype=str)
            if len(a.shape)<2:
                a=np.array([[i.split(":")[0],i.split(":")[1]] for i in a])
            edge=np.vstack((edge,a))
    node_mapping=[{j:i for(i,j) in enumerate(feat.index)}]
    node_mapping=node_mapping[0]
    edge=np.array([[node_mapping[i[0]],node_mapping[i[1]]] for i in edge])
    adj=process_adj(edge,feat.shape[0])
#     pos_pairs=generate_pos_pair(adj.todense(),feat.index,N_WALKS,WALK_LEN)
#     pos_pairs=np.array([[node_mapping[i[0]],node_mapping[i[1]]] for i in pos_pairs])
    nodes=np.arange(feat.shape[0])
#     neg_pairs=np.array(get_negtive_nodes(nodes,dist,adj,N_WALK_LEN,NUM_NEG,dist_aware))
#     dataset=TensorDataset(torch.Tensor(list(node_mapping.values())).int(),torch.cat((torch.zeros(num1),torch.ones(num2))).int())
    dataset=TensorDataset(torch.Tensor(list(nodes)).int())
    return dataset,feat,adj,batch

dataset,feat,adj,batch = load_data(feat_file,edge_file,SEP)
print(dataset)

In [15]:
batch[0]

array([    0,     1,     2, ..., 11023, 11024, 11025])

In [15]:
import importlib
import sys
del sys.modules['spiral.main']
from spiral.main import SPIRAL_integration

In [4]:
SPII=SPIRAL_integration(params,feat_file,edge_file,meta_file)
SPII.train()

--------------------------------
Training.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.24 GiB (GPU 0; 4.00 GiB total capacity; 3.37 GiB already allocated; 0 bytes free; 3.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF